In [ ]:
import sys
import groio
import re
import pandas as pd
import numpy as np

try:
    import io
except ImportError:
    from io import StringIO


class NoGroFile(Exception):
    pass


class ReturnGro(object):
    def __init__(self, title, atoms, box):
        self.title = title.strip()
        self.atoms = atoms
        self.box = box


def gro_parser(gro_file, include_solvent=True):
    if str(gro_file).endswith(".gro"):
        try:
            with open(gro_file, "r") as gf:
                gro_content = gf.read()
        except Exception:
            raise NoGroFile("Check if the .gro path is correct")
        del gro_content
        title, atoms, box = groio.parse_file(gro_file)
        if include_solvent:
            atoms = pd.DataFrame.from_dict(atoms, orient="columns").set_index("atomid")
        else:
            atoms = [d for d in atoms if d.get("resname") not in ["SOL", "NA", "CL"]]
            atoms = groio.renumber(atoms)
            atoms = pd.DataFrame.from_dict(atoms, orient="columns").set_index("atomid")
            title = "Protein without water"

        return ReturnGro(title, atoms, box)


def get_coordinates(gro_return, include_solvent=True):
    gro_parsed = gro_parser(gro_return, include_solvent)
    atom_info = gro_parsed.atoms
    coord_vals = atom_info[["x", "y", "z"]].values

    return coord_vals


'''Bucket list:'''
# Define local coordinate system (We leave it as default from gmx output
# def set_coordinate_center(center_atom, ):

# Calculate coordinates relative to origin

def center_coords(central_atom, gro_return):
    gro_parsed = gro_parser(gro_return)
    atom_info = gro_parsed.atoms


'''if include_water:
     del gro_content
     title, atoms, box = groio.parse_file(gro_file)
 else:
     gro_content = re.sub('\s*[1-9]+SOL\s+[A-Z0-9]+\s+.\.[0-9]{3}\s+\s+.\.[0-9]{3}\s+\s+.\.[0-9]{3}\n', '',
                          gro_content)
     with tempfile.TemporaryFile(mode="w") as gro_tp:
         gro_tp.write(gro_content)
     title, atoms, box = groio.parse_file(gro_tp.gettempdir())'''

if __name__ == '__main__':
    print(get_coordinates(sys.argv[1], include_solvent=True))
    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
